In [1]:
!pip install kaggle


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [3]:

!kaggle datasets download -d ebiswas/imdb-review-dataset

 99% 2.68G/2.69G [00:18<00:00, 111MB/s]
100% 2.69G/2.69G [00:18<00:00, 153MB/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip imdb-review-dataset.zip

Archive:  imdb-review-dataset.zip
  inflating: part-01.json            
  inflating: part-02.json            
  inflating: part-03.json            
  inflating: part-04.json            
  inflating: part-05.json            
  inflating: part-06.json            
  inflating: sample.json             


In [6]:
!git clone https://github.com/vinay20045/json-to-csv 

Cloning into 'json-to-csv'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 67 (delta 2), reused 0 (delta 0), pack-reused 56
Unpacking objects: 100% (67/67), done.


In [7]:
import pandas as pd
import numpy as np
import glob, os

In [8]:
def clean_helpful_values(df):
  df = df.copy()
  a = df[df['helpful'].str[0].str.contains('[,"]')]
  print("Wrong elements with errors to remove in column 0:",len(a))
  b = df[df['helpful'].str[1].str.contains('[,"]')]
  print("Wrong elements with errors to remove in column 1:",len(b))

  a = df['helpful'].str[0].str.replace('[,"]','')
  b = df['helpful'].str[1].str.replace('[,"]','')
  df['helpful'] = pd.Series(list(np.column_stack((a.to_numpy(), b.to_numpy()))))


  a = df[df['helpful'].str[0].str.contains('[,"]')]
  print("Wrong elements with errors to remove in column 0:",len(a))
  b = df[df['helpful'].str[1].str.contains('[,"]')]
  print("Wrong elements with errors to remove in column 1:",len(b))

  return df

def remove_useless_columns(df):
  return df.copy().drop(columns=['reviewer','rating','review_summary'])


def change_date_format(df):
  df = df.copy()
  df['review_date'] = pd.to_datetime(df["review_date"]).dt.strftime('%Y-%m-%d')
  return df

In [9]:

os.chdir("/content")
names = glob.glob("*.json")

!rm -r -f /content/cleaned_files/


!mkdir /content/cleaned_files
!mkdir /content/cleaned_files/cleaned_json
!mkdir /content/cleaned_files/cleaned_csv





In [10]:
for name in names:
  df = pd.read_json("/content/"+name)
  df = remove_useless_columns(df)
  df = change_date_format(df)
  df = clean_helpful_values(df)

  jsonPath ='/content/cleaned_files/cleaned_json/'+name
  df.to_json(jsonPath,orient='records')

  csvPath = '/content/cleaned_files/cleaned_csv/'+name.replace('.json','')+'.csv'
  !python /content/json-to-csv/json_to_csv.py "" $jsonPath $csvPath

Wrong elements with errors to remove in column 0: 13
Wrong elements with errors to remove in column 1: 34


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


Wrong elements with errors to remove in column 0: 0
Wrong elements with errors to remove in column 1: 0
Just completed writing csv file with 7 columns
Wrong elements with errors to remove in column 0: 142
Wrong elements with errors to remove in column 1: 400
Wrong elements with errors to remove in column 0: 0
Wrong elements with errors to remove in column 1: 0
tcmalloc: large alloc 1262272512 bytes == 0x55974bfa6000 @  0x7f531da791e7 0x55974a52e338 0x55974a4f8ad7 0x55974a677575 0x55974a6118b8 0x55974a4fc3a2 0x55974a5dac6e 0x55974a611832 0x55974a4fc3a2 0x55974a4ff5f6 0x55974a67ac03 0x55974a4fbff9 0x55974a5edd4d 0x55974a56fec8 0x55974a56aa2e 0x55974a56a723 0x55974a634812 0x55974a634b8d 0x55974a634a36 0x55974a60c183 0x55974a60be2c 0x7f531c863c87 0x55974a60bd0a
tcmalloc: large alloc 1262272512 bytes == 0x559797372000 @  0x7f531da791e7 0x55974a52e338 0x55974a54366c 0x55974a5d0e13 0x55974a4fbff9 0x55974a4fbef0 0x55974a570123 0x55974a4fd7aa 0x55974a56bb4f 0x55974a56aa2e 0x55974a4fdf21 0x55974

In [11]:
!zip -r '/content/cleaned_files/cleaned_csv/cleaned_csv.zip' '/content/cleaned_files/cleaned_csv/'

  adding: content/cleaned_files/cleaned_csv/ (stored 0%)
  adding: content/cleaned_files/cleaned_csv/cleaned_part-02.csv (deflated 60%)
  adding: content/cleaned_files/cleaned_csv/cleaned_part-04.csv (deflated 61%)
  adding: content/cleaned_files/cleaned_csv/cleaned_part-01.csv (deflated 61%)
  adding: content/cleaned_files/cleaned_csv/cleaned_sample.csv (deflated 60%)
  adding: content/cleaned_files/cleaned_csv/cleaned_part-05.csv (deflated 60%)
  adding: content/cleaned_files/cleaned_csv/cleaned_part-03.csv (deflated 61%)
  adding: content/cleaned_files/cleaned_csv/cleaned_part-06.csv (deflated 63%)


In [14]:
ls -l cleaned_files/cleaned_csv/ --block-size=MB

total 9339MB
-rw-r--r-- 1 root root 2639MB Apr 11 14:54 cleaned_csv.zip
-rw-r--r-- 1 root root  905MB Apr 11 14:43 cleaned_part-01.csv
-rw-r--r-- 1 root root 1090MB Apr 11 14:35 cleaned_part-02.csv
-rw-r--r-- 1 root root 1241MB Apr 11 14:41 cleaned_part-03.csv
-rw-r--r-- 1 root root 1183MB Apr 11 14:33 cleaned_part-04.csv
-rw-r--r-- 1 root root 1577MB Apr 11 14:38 cleaned_part-05.csv
-rw-r--r-- 1 root root  562MB Apr 11 14:31 cleaned_part-06.csv
-rw-r--r-- 1 root root  146MB Apr 11 14:33 cleaned_sample.csv


In [18]:


df = pd.read_csv('/content/cleaned_files/cleaned_csv/cleaned_part-03.csv')
df


,_helpful_0,_helpful_1,_movie,_review_date,_review_detail,_review_id,_spoiler_tag
0,1,3,Dead Man on Campus (1998),2001-12-13,"I didn't even see this movie in the theatres, ...",rw0400504,0
1,2,2,Dead Man on Campus (1998),1999-01-22,It's definitely not worth the money it cost to...,rw0400505,0
2,1,3,Dead Man on Campus (1998),2002-01-09,First of all can I say AT LEAST GET THE PLOT O...,rw0400506,0
3,1,3,Dead Man on Campus (1998),2002-05-01,My friend was ALWAYS telling me about this mov...,rw0400508,0
4,2,3,Dead Man on Campus (1998),1999-01-31,The laughter never stops. This is because the ...,rw0400509,0
...,...,...,...,...,...,...,...
1014995,0,0,The Insider (1999),1999-12-26,"This is an absolute magnificent movie, in fact...",rw0528641,0
1014996,0,0,The Insider (1999),2000-01-07,There are times that The Insider will leave yo...,rw0528642,0
1014997,0,0,The Insider (1999),2000-01-11,The sense of scandal and coverup is in the air...,rw0528643,0
1014998,0,0,The Insider (1999),1999-11-07,"""The Insider"" is the latest from ""Vegas"" and ""...",rw0528644,0


In [22]:
df = pd.read_json('/content/cleaned_files/cleaned_json/cleaned_part-03.json')
df

,review_id,movie,review_date,spoiler_tag,review_detail,helpful
0,rw0400504,Dead Man on Campus (1998),2001-12-13,0,"I didn't even see this movie in the theatres, ...","[1, 3]"
1,rw0400505,Dead Man on Campus (1998),1999-01-22,0,It's definitely not worth the money it cost to...,"[2, 2]"
2,rw0400506,Dead Man on Campus (1998),2002-01-09,0,First of all can I say AT LEAST GET THE PLOT O...,"[1, 3]"
3,rw0400508,Dead Man on Campus (1998),2002-05-01,0,My friend was ALWAYS telling me about this mov...,"[1, 3]"
4,rw0400509,Dead Man on Campus (1998),1999-01-31,0,The laughter never stops. This is because the ...,"[2, 3]"
...,...,...,...,...,...,...
1014995,rw0528641,The Insider (1999),1999-12-26,0,"This is an absolute magnificent movie, in fact...","[0, 0]"
1014996,rw0528642,The Insider (1999),2000-01-07,0,There are times that The Insider will leave yo...,"[0, 0]"
1014997,rw0528643,The Insider (1999),2000-01-11,0,The sense of scandal and coverup is in the air...,"[0, 0]"
1014998,rw0528644,The Insider (1999),1999-11-07,0,"""The Insider"" is the latest from ""Vegas"" and ""...","[0, 0]"


In [21]:
cp /content/cleaned_files/cleaned_csv/cleaned_csv.zip /content/drive/MyDrive/cleaned_imdb_csv.zip